# Module 6 Test: Boto3 and AWS - SOLUTIONS

This test covers the fundamental concepts and practical skills for working with AWS services using the Boto3 library in Python.

## Topics Covered:
- AWS fundamentals and setup
- S3 operations (buckets, objects, upload/download)
- DynamoDB operations
- Lambda and SQS/SNS basics
- Practical patterns (error handling, pagination)

## Instructions:
1. Read each question carefully
2. Write your solution in the provided code cell
3. Since AWS credentials may not be available, many questions use mocking or are conceptual
4. For mock-based questions, use `unittest.mock` or `moto` library patterns
5. Run your code to verify it works (where applicable)

## Grading:
- Questions 1-4: Fundamentals (easier)
- Questions 5-8: Intermediate
- Questions 9-12: Advanced

Good luck!

In [ ]:
# Required imports for this test
import boto3
from botocore.exceptions import ClientError, BotoCoreError
from unittest.mock import Mock, MagicMock, patch
import json
from typing import Dict, List, Optional, Any

---
## Part 1: AWS Fundamentals and Setup
---

### Question 1: Understanding Boto3 Clients vs Resources (Easy)

Boto3 provides two ways to interact with AWS services: **clients** and **resources**.

**Task:** Write a function called `explain_boto3_interfaces()` that returns a dictionary explaining:
1. What a boto3 client is and when to use it
2. What a boto3 resource is and when to use it
3. One advantage of each approach

The function should return a dictionary with keys: `'client'`, `'resource'`, `'client_advantage'`, `'resource_advantage'`

**Example output structure:**
```python
{
    'client': 'description...',
    'resource': 'description...',
    'client_advantage': 'advantage...',
    'resource_advantage': 'advantage...'
}
```

In [ ]:
# SOLUTION for Question 1

def explain_boto3_interfaces() -> Dict[str, str]:
    """
    Explain the differences between boto3 clients and resources.
    
    Returns:
        Dictionary with explanations of client and resource interfaces.
    """
    return {
        'client': (
            "A low-level service interface that maps directly to AWS service APIs. "
            "Clients provide 1:1 mapping to service operations and return dictionary "
            "responses. Use clients when you need access to all service operations or "
            "when resources don't support the operation you need."
        ),
        'resource': (
            "A higher-level, object-oriented interface that represents AWS resources "
            "as Python objects. Resources provide a more Pythonic way to interact with "
            "AWS services. Use resources when you want cleaner, more readable code and "
            "when working with services that have good resource support (S3, EC2, DynamoDB)."
        ),
        'client_advantage': (
            "Complete API coverage - clients provide access to ALL service operations, "
            "including newer features that may not yet be available in resources."
        ),
        'resource_advantage': (
            "More Pythonic and readable code - resources allow chaining operations, "
            "automatic pagination through collections, and working with objects rather "
            "than raw dictionaries."
        )
    }


# Test the function
result = explain_boto3_interfaces()
for key, value in result.items():
    print(f"\n{key.upper()}:")
    print(f"  {value}")

### Question 2: AWS Credential Configuration (Easy)

AWS credentials can be configured in multiple ways with a specific precedence order.

**Task:** Write a function called `get_credential_precedence()` that returns a list of the credential sources boto3 checks, **in order of precedence** (first checked to last checked).

Include at least 5 credential sources.

**Hint:** Think about environment variables, config files, IAM roles, etc.

In [ ]:
# SOLUTION for Question 2

def get_credential_precedence() -> List[str]:
    """
    Return the order of precedence for AWS credential sources.
    
    Boto3 checks these sources in order, using the first valid credentials found.
    
    Returns:
        List of credential sources in order of precedence (first checked to last).
    """
    return [
        "1. Passing credentials as parameters when creating clients/resources",
        "2. Passing credentials as parameters when creating a Session",
        "3. Environment variables (AWS_ACCESS_KEY_ID, AWS_SECRET_ACCESS_KEY, AWS_SESSION_TOKEN)",
        "4. Shared credentials file (~/.aws/credentials)",
        "5. AWS config file (~/.aws/config)",
        "6. Assume Role provider (from config file)",
        "7. Boto2 config file (/etc/boto.cfg and ~/.boto)",
        "8. IAM Role for Amazon EC2 (instance metadata service)",
        "9. IAM Role for ECS tasks (container credentials)",
        "10. IAM Identity Center (SSO) credentials"
    ]


# Test the function
precedence = get_credential_precedence()
print("AWS Credential Precedence (highest to lowest):")
print("=" * 50)
for source in precedence:
    print(source)

---
## Part 2: S3 Operations
---

### Question 3: Creating an S3 Bucket (Easy)

**Task:** Write a function called `create_s3_bucket(bucket_name: str, region: str) -> dict` that:
1. Creates an S3 bucket with the given name in the specified region
2. Handles the special case for `us-east-1` region (no LocationConstraint needed)
3. Returns the response from the create_bucket call
4. Includes proper error handling for `ClientError`

**Note:** You can use mocking to test this function. The function itself should contain the real boto3 code.

In [ ]:
# SOLUTION for Question 3

def create_s3_bucket(bucket_name: str, region: str) -> dict:
    """
    Create an S3 bucket in the specified region.
    
    Args:
        bucket_name: The name of the bucket to create (must be globally unique).
        region: The AWS region where the bucket should be created.
        
    Returns:
        Dictionary containing the response from create_bucket call,
        or error information if the operation failed.
        
    Raises:
        ClientError: If the bucket creation fails.
    """
    s3_client = boto3.client('s3', region_name=region)
    
    try:
        # us-east-1 is special - it doesn't accept LocationConstraint
        if region == 'us-east-1':
            response = s3_client.create_bucket(Bucket=bucket_name)
        else:
            response = s3_client.create_bucket(
                Bucket=bucket_name,
                CreateBucketConfiguration={
                    'LocationConstraint': region
                }
            )
        return response
    
    except ClientError as e:
        error_code = e.response['Error']['Code']
        error_message = e.response['Error']['Message']
        print(f"Error creating bucket: {error_code} - {error_message}")
        raise


# Test with mocking
with patch('boto3.client') as mock_client:
    mock_s3 = MagicMock()
    mock_client.return_value = mock_s3
    mock_s3.create_bucket.return_value = {'Location': '/my-test-bucket'}
    
    # Test us-east-1 (no LocationConstraint)
    result = create_s3_bucket('my-test-bucket', 'us-east-1')
    print(f"us-east-1 result: {result}")
    mock_s3.create_bucket.assert_called_with(Bucket='my-test-bucket')
    
    # Test other region (with LocationConstraint)
    mock_s3.reset_mock()
    result = create_s3_bucket('my-test-bucket', 'eu-west-1')
    print(f"eu-west-1 result: {result}")
    mock_s3.create_bucket.assert_called_with(
        Bucket='my-test-bucket',
        CreateBucketConfiguration={'LocationConstraint': 'eu-west-1'}
    )
    
print("\nAll tests passed!")

### Question 4: Upload and Download Operations (Easy)

**Task:** Write two functions:

1. `upload_file_to_s3(file_path: str, bucket: str, object_key: str) -> bool`
   - Uploads a local file to S3
   - Returns True on success, False on failure
   - Handles errors gracefully

2. `download_file_from_s3(bucket: str, object_key: str, local_path: str) -> bool`
   - Downloads an S3 object to a local file
   - Returns True on success, False on failure
   - Handles errors gracefully

In [ ]:
# SOLUTION for Question 4

def upload_file_to_s3(file_path: str, bucket: str, object_key: str) -> bool:
    """
    Upload a local file to an S3 bucket.
    
    Args:
        file_path: Path to the local file to upload.
        bucket: Name of the S3 bucket.
        object_key: The key (path) for the object in S3.
        
    Returns:
        True if upload was successful, False otherwise.
    """
    s3_client = boto3.client('s3')
    
    try:
        s3_client.upload_file(file_path, bucket, object_key)
        print(f"Successfully uploaded {file_path} to s3://{bucket}/{object_key}")
        return True
    except FileNotFoundError:
        print(f"Error: Local file not found: {file_path}")
        return False
    except ClientError as e:
        print(f"Error uploading file: {e.response['Error']['Message']}")
        return False


def download_file_from_s3(bucket: str, object_key: str, local_path: str) -> bool:
    """
    Download an S3 object to a local file.
    
    Args:
        bucket: Name of the S3 bucket.
        object_key: The key (path) of the object in S3.
        local_path: Path where the file should be saved locally.
        
    Returns:
        True if download was successful, False otherwise.
    """
    s3_client = boto3.client('s3')
    
    try:
        s3_client.download_file(bucket, object_key, local_path)
        print(f"Successfully downloaded s3://{bucket}/{object_key} to {local_path}")
        return True
    except ClientError as e:
        error_code = e.response['Error']['Code']
        if error_code == '404':
            print(f"Error: Object not found: s3://{bucket}/{object_key}")
        else:
            print(f"Error downloading file: {e.response['Error']['Message']}")
        return False


# Test with mocking
with patch('boto3.client') as mock_client:
    mock_s3 = MagicMock()
    mock_client.return_value = mock_s3
    
    # Test successful upload
    result = upload_file_to_s3('/path/to/file.txt', 'my-bucket', 'uploads/file.txt')
    print(f"Upload result: {result}")
    
    # Test successful download
    result = download_file_from_s3('my-bucket', 'uploads/file.txt', '/local/file.txt')
    print(f"Download result: {result}")
    
    # Test failed upload (ClientError)
    mock_s3.upload_file.side_effect = ClientError(
        {'Error': {'Code': 'AccessDenied', 'Message': 'Access Denied'}},
        'upload_file'
    )
    result = upload_file_to_s3('/path/to/file.txt', 'my-bucket', 'uploads/file.txt')
    print(f"Failed upload result: {result}")

### Question 5: Generate Presigned URLs (Intermediate)

Presigned URLs allow temporary access to private S3 objects without requiring AWS credentials.

**Task:** Write a function called `generate_presigned_url(bucket: str, object_key: str, expiration: int = 3600, operation: str = 'get_object') -> Optional[str]` that:
1. Generates a presigned URL for either downloading (`get_object`) or uploading (`put_object`)
2. Uses the specified expiration time in seconds (default 1 hour)
3. Returns the URL string or None if generation fails
4. Handles both `get_object` and `put_object` operations

In [ ]:
# SOLUTION for Question 5

def generate_presigned_url(
    bucket: str,
    object_key: str,
    expiration: int = 3600,
    operation: str = 'get_object'
) -> Optional[str]:
    """
    Generate a presigned URL for S3 operations.
    
    Args:
        bucket: Name of the S3 bucket.
        object_key: The key (path) of the object in S3.
        expiration: URL expiration time in seconds (default: 3600 = 1 hour).
        operation: Either 'get_object' for download or 'put_object' for upload.
        
    Returns:
        The presigned URL string, or None if generation fails.
        
    Raises:
        ValueError: If operation is not 'get_object' or 'put_object'.
    """
    valid_operations = ['get_object', 'put_object']
    if operation not in valid_operations:
        raise ValueError(f"Operation must be one of {valid_operations}")
    
    s3_client = boto3.client('s3')
    
    try:
        url = s3_client.generate_presigned_url(
            ClientMethod=operation,
            Params={
                'Bucket': bucket,
                'Key': object_key
            },
            ExpiresIn=expiration
        )
        return url
    except ClientError as e:
        print(f"Error generating presigned URL: {e.response['Error']['Message']}")
        return None


# Test with mocking
with patch('boto3.client') as mock_client:
    mock_s3 = MagicMock()
    mock_client.return_value = mock_s3
    mock_s3.generate_presigned_url.return_value = 'https://bucket.s3.amazonaws.com/key?signature=xxx'
    
    # Test get_object (download)
    url = generate_presigned_url('my-bucket', 'files/document.pdf')
    print(f"Download URL: {url}")
    
    # Test put_object (upload) with custom expiration
    url = generate_presigned_url('my-bucket', 'uploads/new-file.txt', expiration=7200, operation='put_object')
    print(f"Upload URL: {url}")
    
    # Verify the calls
    calls = mock_s3.generate_presigned_url.call_args_list
    print(f"\nNumber of presigned URL calls: {len(calls)}")
    
    # Test invalid operation
    try:
        generate_presigned_url('my-bucket', 'key', operation='invalid_op')
    except ValueError as e:
        print(f"\nCaught expected error: {e}")

---
## Part 3: DynamoDB Operations
---

### Question 6: DynamoDB CRUD Operations (Intermediate)

**Task:** Create a class called `DynamoDBManager` that provides basic CRUD operations for a DynamoDB table.

The class should have:
1. `__init__(self, table_name: str)` - Initialize with table name and create DynamoDB resource
2. `put_item(self, item: Dict) -> bool` - Add or update an item
3. `get_item(self, key: Dict) -> Optional[Dict]` - Retrieve an item by its key
4. `delete_item(self, key: Dict) -> bool` - Delete an item by its key
5. `scan_table(self) -> List[Dict]` - Return all items in the table

All methods should include proper error handling.

In [ ]:
# SOLUTION for Question 6

class DynamoDBManager:
    """
    A manager class for basic DynamoDB CRUD operations.
    
    Provides a simplified interface for common DynamoDB operations
    including put, get, delete, and scan.
    """
    
    def __init__(self, table_name: str) -> None:
        """
        Initialize the DynamoDB manager.
        
        Args:
            table_name: Name of the DynamoDB table to manage.
        """
        self.table_name = table_name
        self.dynamodb = boto3.resource('dynamodb')
        self.table = self.dynamodb.Table(table_name)
    
    def put_item(self, item: Dict) -> bool:
        """
        Add or update an item in the table.
        
        Args:
            item: Dictionary containing the item attributes.
                  Must include the primary key attribute(s).
                  
        Returns:
            True if successful, False otherwise.
        """
        try:
            self.table.put_item(Item=item)
            return True
        except ClientError as e:
            print(f"Error putting item: {e.response['Error']['Message']}")
            return False
    
    def get_item(self, key: Dict) -> Optional[Dict]:
        """
        Retrieve an item by its primary key.
        
        Args:
            key: Dictionary containing the primary key attribute(s).
            
        Returns:
            The item as a dictionary if found, None otherwise.
        """
        try:
            response = self.table.get_item(Key=key)
            return response.get('Item')
        except ClientError as e:
            print(f"Error getting item: {e.response['Error']['Message']}")
            return None
    
    def delete_item(self, key: Dict) -> bool:
        """
        Delete an item by its primary key.
        
        Args:
            key: Dictionary containing the primary key attribute(s).
            
        Returns:
            True if successful, False otherwise.
        """
        try:
            self.table.delete_item(Key=key)
            return True
        except ClientError as e:
            print(f"Error deleting item: {e.response['Error']['Message']}")
            return False
    
    def scan_table(self) -> List[Dict]:
        """
        Scan and return all items in the table.
        
        Note: Scan reads every item in the table. Use with caution
        on large tables as it can be slow and expensive.
        
        Returns:
            List of all items in the table.
        """
        items = []
        try:
            response = self.table.scan()
            items.extend(response.get('Items', []))
            
            # Handle pagination
            while 'LastEvaluatedKey' in response:
                response = self.table.scan(
                    ExclusiveStartKey=response['LastEvaluatedKey']
                )
                items.extend(response.get('Items', []))
                
            return items
        except ClientError as e:
            print(f"Error scanning table: {e.response['Error']['Message']}")
            return []


# Test with mocking
with patch('boto3.resource') as mock_resource:
    mock_dynamodb = MagicMock()
    mock_table = MagicMock()
    mock_resource.return_value = mock_dynamodb
    mock_dynamodb.Table.return_value = mock_table
    
    # Create manager
    manager = DynamoDBManager('Users')
    
    # Test put_item
    result = manager.put_item({'user_id': '123', 'name': 'John', 'email': 'john@example.com'})
    print(f"Put item result: {result}")
    
    # Test get_item
    mock_table.get_item.return_value = {
        'Item': {'user_id': '123', 'name': 'John', 'email': 'john@example.com'}
    }
    item = manager.get_item({'user_id': '123'})
    print(f"Get item result: {item}")
    
    # Test delete_item
    result = manager.delete_item({'user_id': '123'})
    print(f"Delete item result: {result}")
    
    # Test scan_table
    mock_table.scan.return_value = {
        'Items': [
            {'user_id': '1', 'name': 'Alice'},
            {'user_id': '2', 'name': 'Bob'}
        ]
    }
    items = manager.scan_table()
    print(f"Scan result: {items}")

### Question 7: DynamoDB Query with Filter (Intermediate)

**Task:** Write a function called `query_users_by_status(table_name: str, status: str, min_age: int) -> List[Dict]` that:
1. Queries a DynamoDB table named `Users` with a GSI (Global Secondary Index) on `status`
2. Filters results to only include users with age >= min_age
3. Uses KeyConditionExpression for the status query
4. Uses FilterExpression for the age filter
5. Returns the list of matching items

**Assume the table has:**
- Primary key: `user_id` (String)
- GSI: `status-index` on `status` attribute
- Attributes: `user_id`, `name`, `status`, `age`

In [ ]:
# SOLUTION for Question 7

from boto3.dynamodb.conditions import Key, Attr


def query_users_by_status(table_name: str, status: str, min_age: int) -> List[Dict]:
    """
    Query users by status and filter by minimum age.
    
    Uses a GSI on the status attribute for efficient querying,
    then applies a filter expression for the age condition.
    
    Args:
        table_name: Name of the DynamoDB table.
        status: The status value to query for (e.g., 'active', 'inactive').
        min_age: Minimum age filter (inclusive).
        
    Returns:
        List of user items matching the criteria.
    """
    dynamodb = boto3.resource('dynamodb')
    table = dynamodb.Table(table_name)
    
    items = []
    
    try:
        # Query using the GSI with a filter expression
        response = table.query(
            IndexName='status-index',
            KeyConditionExpression=Key('status').eq(status),
            FilterExpression=Attr('age').gte(min_age)
        )
        items.extend(response.get('Items', []))
        
        # Handle pagination
        while 'LastEvaluatedKey' in response:
            response = table.query(
                IndexName='status-index',
                KeyConditionExpression=Key('status').eq(status),
                FilterExpression=Attr('age').gte(min_age),
                ExclusiveStartKey=response['LastEvaluatedKey']
            )
            items.extend(response.get('Items', []))
        
        return items
        
    except ClientError as e:
        print(f"Error querying table: {e.response['Error']['Message']}")
        return []


# Test with mocking
with patch('boto3.resource') as mock_resource:
    mock_dynamodb = MagicMock()
    mock_table = MagicMock()
    mock_resource.return_value = mock_dynamodb
    mock_dynamodb.Table.return_value = mock_table
    
    # Mock the query response
    mock_table.query.return_value = {
        'Items': [
            {'user_id': '1', 'name': 'Alice', 'status': 'active', 'age': 30},
            {'user_id': '2', 'name': 'Bob', 'status': 'active', 'age': 25},
        ]
    }
    
    # Test the function
    users = query_users_by_status('Users', 'active', 21)
    print(f"Found {len(users)} users:")
    for user in users:
        print(f"  - {user['name']} (age: {user['age']})")
    
    # Verify the query call
    mock_table.query.assert_called()

---
## Part 4: Lambda and Messaging Services
---

### Question 8: Invoke Lambda Function (Intermediate)

**Task:** Write a function called `invoke_lambda(function_name: str, payload: Dict, invocation_type: str = 'RequestResponse') -> Dict` that:
1. Invokes an AWS Lambda function with the given payload
2. Supports both synchronous (`RequestResponse`) and asynchronous (`Event`) invocation types
3. For synchronous calls, parses and returns the response payload as a dictionary
4. For async calls, returns a dictionary with the status code
5. Handles errors and returns an error dictionary on failure

**Return format:**
- Success (sync): The parsed response payload
- Success (async): `{'status': 'accepted', 'status_code': 202}`
- Error: `{'error': 'error message'}`

In [ ]:
# SOLUTION for Question 8

def invoke_lambda(
    function_name: str,
    payload: Dict,
    invocation_type: str = 'RequestResponse'
) -> Dict:
    """
    Invoke an AWS Lambda function.
    
    Args:
        function_name: Name or ARN of the Lambda function.
        payload: Dictionary to send as the event payload.
        invocation_type: 'RequestResponse' for sync, 'Event' for async.
        
    Returns:
        For sync: The parsed response payload.
        For async: Status dictionary with 'status' and 'status_code'.
        On error: Dictionary with 'error' key.
    """
    if invocation_type not in ['RequestResponse', 'Event']:
        return {'error': f"Invalid invocation type: {invocation_type}"}
    
    lambda_client = boto3.client('lambda')
    
    try:
        response = lambda_client.invoke(
            FunctionName=function_name,
            InvocationType=invocation_type,
            Payload=json.dumps(payload)
        )
        
        status_code = response['StatusCode']
        
        # Async invocation
        if invocation_type == 'Event':
            return {
                'status': 'accepted',
                'status_code': status_code
            }
        
        # Sync invocation - parse the response
        response_payload = response['Payload'].read().decode('utf-8')
        
        # Check for function error
        if 'FunctionError' in response:
            return {'error': f"Function error: {response_payload}"}
        
        # Parse and return the response
        try:
            return json.loads(response_payload)
        except json.JSONDecodeError:
            return {'response': response_payload}
            
    except ClientError as e:
        return {'error': e.response['Error']['Message']}
    except Exception as e:
        return {'error': str(e)}


# Test with mocking
from io import BytesIO

with patch('boto3.client') as mock_client:
    mock_lambda = MagicMock()
    mock_client.return_value = mock_lambda
    
    # Test synchronous invocation
    mock_response_payload = BytesIO(json.dumps({'result': 'success', 'data': [1, 2, 3]}).encode())
    mock_lambda.invoke.return_value = {
        'StatusCode': 200,
        'Payload': mock_response_payload
    }
    
    result = invoke_lambda('my-function', {'key': 'value'})
    print(f"Sync invocation result: {result}")
    
    # Test async invocation
    mock_lambda.invoke.return_value = {
        'StatusCode': 202,
        'Payload': BytesIO(b'')
    }
    
    result = invoke_lambda('my-function', {'key': 'value'}, 'Event')
    print(f"Async invocation result: {result}")
    
    # Test error handling
    mock_lambda.invoke.side_effect = ClientError(
        {'Error': {'Code': 'ResourceNotFoundException', 'Message': 'Function not found'}},
        'invoke'
    )
    
    result = invoke_lambda('non-existent-function', {})
    print(f"Error result: {result}")

### Question 9: SQS Message Queue Operations (Advanced)

**Task:** Create a class called `SQSManager` that handles SQS operations:

1. `__init__(self, queue_url: str)` - Initialize with queue URL
2. `send_message(self, body: str, attributes: Optional[Dict] = None) -> Optional[str]` - Send a message, return message ID
3. `send_batch(self, messages: List[Dict]) -> Dict` - Send up to 10 messages in a batch, return success/failure counts
4. `receive_messages(self, max_count: int = 1, wait_time: int = 0) -> List[Dict]` - Receive messages with long polling support
5. `delete_message(self, receipt_handle: str) -> bool` - Delete a processed message

**Note:** The batch send should handle the 10-message limit and return `{'successful': count, 'failed': count}`

In [ ]:
# SOLUTION for Question 9

import uuid


class SQSManager:
    """
    A manager class for SQS queue operations.
    
    Provides methods for sending, receiving, and deleting messages
    from an SQS queue, including batch operations.
    """
    
    MAX_BATCH_SIZE = 10  # SQS limit for batch operations
    
    def __init__(self, queue_url: str) -> None:
        """
        Initialize the SQS manager.
        
        Args:
            queue_url: The URL of the SQS queue.
        """
        self.queue_url = queue_url
        self.sqs_client = boto3.client('sqs')
    
    def send_message(
        self,
        body: str,
        attributes: Optional[Dict] = None
    ) -> Optional[str]:
        """
        Send a single message to the queue.
        
        Args:
            body: The message body (string).
            attributes: Optional message attributes dictionary.
            
        Returns:
            The message ID if successful, None otherwise.
        """
        try:
            params = {
                'QueueUrl': self.queue_url,
                'MessageBody': body
            }
            
            if attributes:
                params['MessageAttributes'] = attributes
            
            response = self.sqs_client.send_message(**params)
            return response.get('MessageId')
            
        except ClientError as e:
            print(f"Error sending message: {e.response['Error']['Message']}")
            return None
    
    def send_batch(self, messages: List[Dict]) -> Dict:
        """
        Send multiple messages in a batch (up to 10).
        
        Args:
            messages: List of message dictionaries with 'body' key
                     and optional 'attributes' key.
                     
        Returns:
            Dictionary with 'successful' and 'failed' counts.
        """
        if not messages:
            return {'successful': 0, 'failed': 0}
        
        # Limit to MAX_BATCH_SIZE messages
        batch = messages[:self.MAX_BATCH_SIZE]
        
        entries = []
        for i, msg in enumerate(batch):
            entry = {
                'Id': str(i),
                'MessageBody': msg.get('body', '')
            }
            if 'attributes' in msg:
                entry['MessageAttributes'] = msg['attributes']
            entries.append(entry)
        
        try:
            response = self.sqs_client.send_message_batch(
                QueueUrl=self.queue_url,
                Entries=entries
            )
            
            successful = len(response.get('Successful', []))
            failed = len(response.get('Failed', []))
            
            return {'successful': successful, 'failed': failed}
            
        except ClientError as e:
            print(f"Error in batch send: {e.response['Error']['Message']}")
            return {'successful': 0, 'failed': len(batch)}
    
    def receive_messages(
        self,
        max_count: int = 1,
        wait_time: int = 0
    ) -> List[Dict]:
        """
        Receive messages from the queue.
        
        Args:
            max_count: Maximum number of messages to receive (1-10).
            wait_time: Long polling wait time in seconds (0-20).
            
        Returns:
            List of message dictionaries with 'body', 'message_id',
            and 'receipt_handle' keys.
        """
        # Ensure valid ranges
        max_count = min(max(1, max_count), 10)
        wait_time = min(max(0, wait_time), 20)
        
        try:
            response = self.sqs_client.receive_message(
                QueueUrl=self.queue_url,
                MaxNumberOfMessages=max_count,
                WaitTimeSeconds=wait_time,
                MessageAttributeNames=['All']
            )
            
            messages = []
            for msg in response.get('Messages', []):
                messages.append({
                    'body': msg['Body'],
                    'message_id': msg['MessageId'],
                    'receipt_handle': msg['ReceiptHandle'],
                    'attributes': msg.get('MessageAttributes', {})
                })
            
            return messages
            
        except ClientError as e:
            print(f"Error receiving messages: {e.response['Error']['Message']}")
            return []
    
    def delete_message(self, receipt_handle: str) -> bool:
        """
        Delete a message from the queue.
        
        Args:
            receipt_handle: The receipt handle from receive_message.
            
        Returns:
            True if deletion was successful, False otherwise.
        """
        try:
            self.sqs_client.delete_message(
                QueueUrl=self.queue_url,
                ReceiptHandle=receipt_handle
            )
            return True
        except ClientError as e:
            print(f"Error deleting message: {e.response['Error']['Message']}")
            return False


# Test with mocking
with patch('boto3.client') as mock_client:
    mock_sqs = MagicMock()
    mock_client.return_value = mock_sqs
    
    manager = SQSManager('https://sqs.us-east-1.amazonaws.com/123456789012/my-queue')
    
    # Test send_message
    mock_sqs.send_message.return_value = {'MessageId': 'msg-123'}
    msg_id = manager.send_message('Hello, World!')
    print(f"Sent message ID: {msg_id}")
    
    # Test send_batch
    mock_sqs.send_message_batch.return_value = {
        'Successful': [{'Id': '0'}, {'Id': '1'}],
        'Failed': []
    }
    batch_result = manager.send_batch([
        {'body': 'Message 1'},
        {'body': 'Message 2'}
    ])
    print(f"Batch result: {batch_result}")
    
    # Test receive_messages
    mock_sqs.receive_message.return_value = {
        'Messages': [
            {
                'Body': 'Test message',
                'MessageId': 'msg-456',
                'ReceiptHandle': 'handle-789'
            }
        ]
    }
    messages = manager.receive_messages(max_count=5, wait_time=10)
    print(f"Received {len(messages)} message(s)")
    
    # Test delete_message
    result = manager.delete_message('handle-789')
    print(f"Delete result: {result}")

---
## Part 5: Practical Patterns
---

### Question 10: Pagination Handler (Advanced)

Many AWS operations return paginated results. Boto3 provides paginators to handle this.

**Task:** Write a function called `list_all_s3_objects(bucket: str, prefix: str = '') -> List[Dict]` that:
1. Uses a paginator to list ALL objects in an S3 bucket (handling pagination automatically)
2. Filters by the given prefix (if provided)
3. Returns a list of dictionaries with `key`, `size`, and `last_modified` for each object
4. Handles empty buckets gracefully (returns empty list)

**Hint:** Use `s3_client.get_paginator('list_objects_v2')`

In [ ]:
# SOLUTION for Question 10

from datetime import datetime


def list_all_s3_objects(bucket: str, prefix: str = '') -> List[Dict]:
    """
    List all objects in an S3 bucket using pagination.
    
    Args:
        bucket: Name of the S3 bucket.
        prefix: Optional prefix to filter objects.
        
    Returns:
        List of dictionaries with 'key', 'size', and 'last_modified'
        for each object.
    """
    s3_client = boto3.client('s3')
    objects = []
    
    try:
        # Create a paginator for list_objects_v2
        paginator = s3_client.get_paginator('list_objects_v2')
        
        # Configure pagination parameters
        page_config = {
            'Bucket': bucket
        }
        if prefix:
            page_config['Prefix'] = prefix
        
        # Iterate through all pages
        for page in paginator.paginate(**page_config):
            # Handle empty bucket or no matching objects
            if 'Contents' not in page:
                continue
            
            for obj in page['Contents']:
                objects.append({
                    'key': obj['Key'],
                    'size': obj['Size'],
                    'last_modified': obj['LastModified']
                })
        
        return objects
        
    except ClientError as e:
        print(f"Error listing objects: {e.response['Error']['Message']}")
        return []


# Test with mocking
with patch('boto3.client') as mock_client:
    mock_s3 = MagicMock()
    mock_client.return_value = mock_s3
    
    # Create a mock paginator
    mock_paginator = MagicMock()
    mock_s3.get_paginator.return_value = mock_paginator
    
    # Mock paginated results (simulating 2 pages)
    mock_paginator.paginate.return_value = [
        {
            'Contents': [
                {'Key': 'file1.txt', 'Size': 1024, 'LastModified': datetime(2024, 1, 1)},
                {'Key': 'file2.txt', 'Size': 2048, 'LastModified': datetime(2024, 1, 2)},
            ]
        },
        {
            'Contents': [
                {'Key': 'folder/file3.txt', 'Size': 512, 'LastModified': datetime(2024, 1, 3)},
            ]
        }
    ]
    
    # Test listing all objects
    objects = list_all_s3_objects('my-bucket')
    print(f"Found {len(objects)} objects:")
    for obj in objects:
        print(f"  - {obj['key']} ({obj['size']} bytes)")
    
    # Test with prefix
    mock_paginator.paginate.return_value = [
        {
            'Contents': [
                {'Key': 'folder/file3.txt', 'Size': 512, 'LastModified': datetime(2024, 1, 3)},
            ]
        }
    ]
    
    objects = list_all_s3_objects('my-bucket', prefix='folder/')
    print(f"\nFound {len(objects)} objects with prefix 'folder/':")
    for obj in objects:
        print(f"  - {obj['key']}")
    
    # Test empty bucket
    mock_paginator.paginate.return_value = [{}]  # No Contents key
    objects = list_all_s3_objects('empty-bucket')
    print(f"\nEmpty bucket result: {objects}")

### Question 11: Retry Logic with Exponential Backoff (Advanced)

AWS operations can fail due to throttling or transient errors. Implementing proper retry logic is essential.

**Task:** Write a decorator called `aws_retry` that:
1. Retries a function up to `max_retries` times (default 3)
2. Uses exponential backoff: wait time = `base_delay * (2 ** attempt)` seconds
3. Only retries on specific exceptions: `ClientError` with throttling error codes (`Throttling`, `ThrottlingException`, `RequestLimitExceeded`)
4. Re-raises the exception after all retries are exhausted
5. Logs each retry attempt (print is fine for this exercise)

**Usage example:**
```python
@aws_retry(max_retries=3, base_delay=1)
def my_aws_operation():
    # AWS code here
    pass
```

In [ ]:
# SOLUTION for Question 11

import time
from functools import wraps
from typing import Callable


def aws_retry(max_retries: int = 3, base_delay: float = 1.0) -> Callable:
    """
    Decorator for retrying AWS operations with exponential backoff.
    
    Only retries on throttling-related ClientError exceptions.
    
    Args:
        max_retries: Maximum number of retry attempts.
        base_delay: Base delay in seconds for exponential backoff.
        
    Returns:
        Decorated function with retry logic.
    """
    # Error codes that indicate throttling
    THROTTLING_CODES = {
        'Throttling',
        'ThrottlingException',
        'RequestLimitExceeded',
        'ProvisionedThroughputExceededException',
        'TooManyRequestsException'
    }
    
    def decorator(func: Callable) -> Callable:
        @wraps(func)
        def wrapper(*args, **kwargs):
            last_exception = None
            
            for attempt in range(max_retries + 1):
                try:
                    return func(*args, **kwargs)
                    
                except ClientError as e:
                    error_code = e.response['Error']['Code']
                    
                    # Only retry on throttling errors
                    if error_code not in THROTTLING_CODES:
                        raise
                    
                    last_exception = e
                    
                    # If we've exhausted all retries, raise
                    if attempt >= max_retries:
                        print(f"Max retries ({max_retries}) exhausted for {func.__name__}")
                        raise
                    
                    # Calculate wait time with exponential backoff
                    wait_time = base_delay * (2 ** attempt)
                    
                    print(
                        f"Retry {attempt + 1}/{max_retries} for {func.__name__} "
                        f"after {error_code}. Waiting {wait_time:.2f}s..."
                    )
                    
                    time.sleep(wait_time)
            
            # This shouldn't be reached, but just in case
            if last_exception:
                raise last_exception
                
        return wrapper
    return decorator


# Test the decorator
call_count = 0

@aws_retry(max_retries=3, base_delay=0.1)  # Short delay for testing
def mock_throttled_operation():
    """Simulates an operation that gets throttled."""
    global call_count
    call_count += 1
    
    # Succeed on the 3rd attempt
    if call_count < 3:
        raise ClientError(
            {'Error': {'Code': 'Throttling', 'Message': 'Rate exceeded'}},
            'TestOperation'
        )
    return 'Success!'


@aws_retry(max_retries=2, base_delay=0.1)
def mock_always_fails():
    """Simulates an operation that always fails with throttling."""
    raise ClientError(
        {'Error': {'Code': 'ThrottlingException', 'Message': 'Too many requests'}},
        'TestOperation'
    )


@aws_retry(max_retries=3, base_delay=0.1)
def mock_non_throttling_error():
    """Simulates a non-throttling error (should not retry)."""
    raise ClientError(
        {'Error': {'Code': 'AccessDenied', 'Message': 'Access Denied'}},
        'TestOperation'
    )


# Test 1: Operation succeeds after retries
print("Test 1: Operation with retries that eventually succeeds")
call_count = 0
result = mock_throttled_operation()
print(f"Result: {result}")
print(f"Total calls: {call_count}\n")

# Test 2: Operation fails after all retries
print("Test 2: Operation that fails after all retries")
try:
    mock_always_fails()
except ClientError as e:
    print(f"Caught expected error: {e.response['Error']['Code']}\n")

# Test 3: Non-throttling error (should not retry)
print("Test 3: Non-throttling error (should not retry)")
try:
    mock_non_throttling_error()
except ClientError as e:
    print(f"Caught non-throttling error immediately: {e.response['Error']['Code']}")

### Question 12: Multi-Service Integration (Advanced)

**Scenario:** You need to build a simple data pipeline that:
1. Receives a file upload notification
2. Processes the file metadata
3. Stores results in DynamoDB
4. Sends a notification when complete

**Task:** Write a class called `S3EventProcessor` with the following methods:

1. `__init__(self, dynamodb_table: str, sns_topic_arn: str)` - Initialize with DynamoDB table name and SNS topic ARN

2. `process_s3_event(self, event: Dict) -> Dict` - Process an S3 event notification:
   - Extract bucket name, object key, and size from the event
   - Store metadata in DynamoDB (object_key as primary key, include bucket, size, processed_at timestamp)
   - Publish a success notification to SNS
   - Return a summary dict with status and processed object info

3. `_store_metadata(self, metadata: Dict) -> bool` - Helper to store in DynamoDB

4. `_send_notification(self, message: str, subject: str) -> bool` - Helper to publish to SNS

**Sample S3 event structure (simplified):**
```python
{
    'Records': [{
        's3': {
            'bucket': {'name': 'my-bucket'},
            'object': {'key': 'path/to/file.txt', 'size': 1024}
        }
    }]
}
```

In [ ]:
# SOLUTION for Question 12

from datetime import datetime, timezone


class S3EventProcessor:
    """
    Processes S3 events and stores metadata in DynamoDB with SNS notifications.
    
    This class demonstrates a common serverless pattern where S3 events
    trigger processing that involves multiple AWS services.
    """
    
    def __init__(self, dynamodb_table: str, sns_topic_arn: str) -> None:
        """
        Initialize the S3 event processor.
        
        Args:
            dynamodb_table: Name of the DynamoDB table for storing metadata.
            sns_topic_arn: ARN of the SNS topic for notifications.
        """
        self.dynamodb_table = dynamodb_table
        self.sns_topic_arn = sns_topic_arn
        
        # Initialize AWS clients
        self.dynamodb = boto3.resource('dynamodb')
        self.table = self.dynamodb.Table(dynamodb_table)
        self.sns_client = boto3.client('sns')
    
    def process_s3_event(self, event: Dict) -> Dict:
        """
        Process an S3 event notification.
        
        Extracts file metadata from the event, stores it in DynamoDB,
        and sends an SNS notification.
        
        Args:
            event: S3 event notification dictionary.
            
        Returns:
            Summary dictionary with:
            - status: 'success' or 'error'
            - processed_objects: List of processed object info
            - error: Error message (if status is 'error')
        """
        processed_objects = []
        errors = []
        
        try:
            records = event.get('Records', [])
            
            if not records:
                return {
                    'status': 'error',
                    'error': 'No records found in event',
                    'processed_objects': []
                }
            
            for record in records:
                try:
                    # Extract S3 information
                    s3_info = record.get('s3', {})
                    bucket_name = s3_info.get('bucket', {}).get('name')
                    object_key = s3_info.get('object', {}).get('key')
                    object_size = s3_info.get('object', {}).get('size', 0)
                    
                    if not bucket_name or not object_key:
                        errors.append(f"Missing bucket or object key in record")
                        continue
                    
                    # Create metadata record
                    timestamp = datetime.now(timezone.utc).isoformat()
                    metadata = {
                        'object_key': object_key,
                        'bucket': bucket_name,
                        'size': object_size,
                        'processed_at': timestamp
                    }
                    
                    # Store metadata in DynamoDB
                    if not self._store_metadata(metadata):
                        errors.append(f"Failed to store metadata for {object_key}")
                        continue
                    
                    # Send notification
                    notification_message = (
                        f"Successfully processed file: {object_key}\n"
                        f"Bucket: {bucket_name}\n"
                        f"Size: {object_size} bytes\n"
                        f"Processed at: {timestamp}"
                    )
                    self._send_notification(
                        message=notification_message,
                        subject=f"S3 File Processed: {object_key}"
                    )
                    
                    processed_objects.append({
                        'bucket': bucket_name,
                        'key': object_key,
                        'size': object_size
                    })
                    
                except Exception as e:
                    errors.append(f"Error processing record: {str(e)}")
            
            # Determine overall status
            if errors and not processed_objects:
                return {
                    'status': 'error',
                    'error': '; '.join(errors),
                    'processed_objects': []
                }
            elif errors:
                return {
                    'status': 'partial_success',
                    'processed_objects': processed_objects,
                    'errors': errors
                }
            else:
                return {
                    'status': 'success',
                    'processed_objects': processed_objects
                }
                
        except Exception as e:
            return {
                'status': 'error',
                'error': str(e),
                'processed_objects': []
            }
    
    def _store_metadata(self, metadata: Dict) -> bool:
        """
        Store file metadata in DynamoDB.
        
        Args:
            metadata: Dictionary containing file metadata.
            
        Returns:
            True if storage was successful, False otherwise.
        """
        try:
            self.table.put_item(Item=metadata)
            return True
        except ClientError as e:
            print(f"DynamoDB error: {e.response['Error']['Message']}")
            return False
    
    def _send_notification(self, message: str, subject: str) -> bool:
        """
        Publish a notification to SNS.
        
        Args:
            message: The notification message body.
            subject: The notification subject.
            
        Returns:
            True if notification was sent successfully, False otherwise.
        """
        try:
            self.sns_client.publish(
                TopicArn=self.sns_topic_arn,
                Message=message,
                Subject=subject
            )
            return True
        except ClientError as e:
            print(f"SNS error: {e.response['Error']['Message']}")
            return False


# Test with mocking
with patch('boto3.resource') as mock_resource, \
     patch('boto3.client') as mock_client:
    
    # Setup mocks
    mock_dynamodb = MagicMock()
    mock_table = MagicMock()
    mock_sns = MagicMock()
    
    mock_resource.return_value = mock_dynamodb
    mock_dynamodb.Table.return_value = mock_table
    mock_client.return_value = mock_sns
    
    # Create processor
    processor = S3EventProcessor(
        dynamodb_table='FileMetadata',
        sns_topic_arn='arn:aws:sns:us-east-1:123456789012:file-notifications'
    )
    
    # Test S3 event
    test_event = {
        'Records': [
            {
                's3': {
                    'bucket': {'name': 'my-bucket'},
                    'object': {'key': 'uploads/document.pdf', 'size': 102400}
                }
            },
            {
                's3': {
                    'bucket': {'name': 'my-bucket'},
                    'object': {'key': 'uploads/image.png', 'size': 51200}
                }
            }
        ]
    }
    
    result = processor.process_s3_event(test_event)
    
    print(f"Processing result:")
    print(f"  Status: {result['status']}")
    print(f"  Processed objects: {len(result['processed_objects'])}")
    for obj in result['processed_objects']:
        print(f"    - {obj['key']} ({obj['size']} bytes)")
    
    # Verify DynamoDB was called
    assert mock_table.put_item.call_count == 2, "Should store 2 items"
    
    # Verify SNS was called
    assert mock_sns.publish.call_count == 2, "Should send 2 notifications"
    
    print("\nAll assertions passed!")
    
    # Test with empty event
    empty_result = processor.process_s3_event({'Records': []})
    print(f"\nEmpty event result: {empty_result['status']}")

---
## End of Test - SOLUTIONS

This solutions notebook demonstrates:
- Proper type hints in all function signatures
- Comprehensive error handling for AWS operations
- Clear, readable code following PEP 8
- Effective use of mocking for testing AWS code without real credentials
- Best practices for boto3 usage including pagination and retry logic
---